In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 69kB/s 
     |████████████████████████████████| 204kB 26.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=8496f1312d859f03fa209a34512670be1518fba46db29e617905808a98222a87
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


### Create Resilient Distributed Datasets (RDD)

##### Method 1
using *parallelize()* function

In [2]:
from pyspark.sql import SparkSession

# Creating a Spark Session
spark = SparkSession \
.builder \
.appName("Python Spark create RDD example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()




In [3]:
# Spark Details
spark

In [4]:
# Creating RDD using parallelize function

df = spark.sparkContext.parallelize([(1, 2, 3, 'a b c'),
(4, 5, 6, 'd e f'),
(7, 8, 9, 'g h i')]).toDF(['col1', 'col2', 'col3','col4'])


In [5]:
df.show()

+----+----+----+-----+
|col1|col2|col3| col4|
+----+----+----+-----+
|   1|   2|   3|a b c|
|   4|   5|   6|d e f|
|   7|   8|   9|g h i|
+----+----+----+-----+



In [6]:
myData = spark.sparkContext.parallelize([(1,2), (3,4), (5,6), (7,8), (9,10)])
myData.collect()

[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]

##### Method 2
using *createDateFrame()* function

In [7]:
Employee = spark.createDataFrame([
('1', 'Joe', '70000', '1'),
('2', 'Henry', '80000', '2'),
('3', 'Sam', '60000', '2'),
('4', 'Max', '90000', '1')],
['Id', 'Name', 'Sallary','DepartmentId']
)


In [8]:
Employee.show()

+---+-----+-------+------------+
| Id| Name|Sallary|DepartmentId|
+---+-----+-------+------------+
|  1|  Joe|  70000|           1|
|  2|Henry|  80000|           2|
|  3|  Sam|  60000|           2|
|  4|  Max|  90000|           1|
+---+-----+-------+------------+



##### Method 3
using *read* and *load* functions

The data for this part is used from FORCE 2020 Machine Learning Contest with Wells and Seismic

(https://www.npd.no/en/force/events/machine-learning-contest-with-wells-and-seismic/)

To know more about the data, please visit: https://xeek.ai/challenges/force-well-logs/overview

In [9]:
!pip install gdown

In [10]:
! gdown -O log_data.csv https://drive.google.com/uc?id=1YVq71iUjSjqyqJtkmSRbC9NLMCrx0478

Downloading...
From: https://drive.google.com/uc?id=1YVq71iUjSjqyqJtkmSRbC9NLMCrx0478
To: /content/log_data.csv
280MB [00:05, 48.5MB/s]


In [32]:
df = spark.read.format('csv').options(header='true', inferschema='true').load("log_data.csv",header=True, sep = ';')
df.show(5)
df.printSchema()


+-------+--------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+------------+---------+----+----+----+--------------------------------+---------------------------------+
|   WELL|DEPTH_MD|       X_LOC|    Y_LOC|       Z_LOC|       GROUP|FORMATION|        CALI|RSHA|        RMED|        RDEP|        RHOB|          GR| SGR|NPHI|         PEF|         DTC|          SP|  BS|         ROP| DTS|DCAL|        DRHO|MUDWEIGHT|RMIC|ROPA| RXO|FORCE_2020_LITHOFACIES_LITHOLOGY|FORCE_2020_LITHOFACIES_CONFIDENCE|
+-------+--------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+------------+---------+----+----+----+--------------------------------+---------------------------------+
|15/9-13| 

In [12]:
print (df.head(4))

print (df.columns)

print (df.dtypes)

[Row(WELL='15/9-13', DEPTH_MD=494.528, X_LOC=437641.96875, Y_LOC=6470972.5, Z_LOC=-469.5018311, GROUP='NORDLAND GP.', FORMATION=None, CALI=19.480834961, RSHA=None, RMED=1.6114097834, RDEP=1.7986813784, RHOB=1.884185791, GR=80.20085144, SGR=None, NPHI=None, PEF=20.915468216, DTC=161.13117981, SP=24.612379074, BS=None, ROP=34.63640976, DTS=None, DCAL=None, DRHO=-0.574927628, MUDWEIGHT=None, RMIC=None, ROPA=None, RXO=None, FORCE_2020_LITHOFACIES_LITHOLOGY=65000, FORCE_2020_LITHOFACIES_CONFIDENCE=1.0), Row(WELL='15/9-13', DEPTH_MD=494.68, X_LOC=437641.96875, Y_LOC=6470972.5, Z_LOC=-469.6538086, GROUP='NORDLAND GP.', FORMATION=None, CALI=19.468799591, RSHA=None, RMED=1.6180702448, RDEP=1.7956413031, RHOB=1.8897935152, GR=79.262886047, SGR=None, NPHI=None, PEF=19.383012772, DTC=160.60346985, SP=23.895530701, BS=None, ROP=34.63640976, DTS=None, DCAL=None, DRHO=-0.570188403, MUDWEIGHT=None, RMIC=None, ROPA=None, RXO=None, FORCE_2020_LITHOFACIES_LITHOLOGY=65000, FORCE_2020_LITHOFACIES_CONFIDENC

In [13]:
print('Shape of Pandas: ',df.toPandas().shape)

Shape of Pandas:  (1170511, 29)


In [14]:
# Finding null values in a column
from pyspark.sql.functions import col
df.where(col("NPHI").isNull()).toPandas().shape



(405102, 29)

In [15]:
# Renaming Column Names
mapping = {'FORMATION':'formation','DEPTH_MD':'depth'}
new_names = [mapping.get(col,col) for col in df.columns]

df.toDF(*new_names).show(4)


+-------+-------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+------------+---------+----+----+----+--------------------------------+---------------------------------+
|   WELL|  depth|       X_LOC|    Y_LOC|       Z_LOC|       GROUP|formation|        CALI|RSHA|        RMED|        RDEP|        RHOB|          GR| SGR|NPHI|         PEF|         DTC|          SP|  BS|         ROP| DTS|DCAL|        DRHO|MUDWEIGHT|RMIC|ROPA| RXO|FORCE_2020_LITHOFACIES_LITHOLOGY|FORCE_2020_LITHOFACIES_CONFIDENCE|
+-------+-------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+------------+---------+----+----+----+--------------------------------+---------------------------------+
|15/9-13|494.

In [16]:
# Dropping Column Names

drop_columns = ['DRHO','RMIC']
df.drop(*drop_columns).show(4)

+-------+--------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+---------+----+----+--------------------------------+---------------------------------+
|   WELL|DEPTH_MD|       X_LOC|    Y_LOC|       Z_LOC|       GROUP|FORMATION|        CALI|RSHA|        RMED|        RDEP|        RHOB|          GR| SGR|NPHI|         PEF|         DTC|          SP|  BS|         ROP| DTS|DCAL|MUDWEIGHT|ROPA| RXO|FORCE_2020_LITHOFACIES_LITHOLOGY|FORCE_2020_LITHOFACIES_CONFIDENCE|
+-------+--------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+---------+----+----+--------------------------------+---------------------------------+
|15/9-13| 494.528|437641.96875|6470972.5|-469.5018311|NORDLAND G

In [17]:
# Unique Values in Well Column

df.select("WELL").distinct().show()


+----------+
|      WELL|
+----------+
| 16/2-11 A|
|    16/7-4|
|    31/3-3|
|    33/5-2|
|  34/10-21|
|25/2-13 T4|
|    31/3-1|
|    16/4-1|
|    25/9-1|
|    26/4-1|
|    33/9-1|
| 34/11-2 S|
|   33/9-17|
|   25/11-5|
|    25/3-1|
|  33/6-3 S|
|   16/10-1|
|   16/10-2|
|  34/3-1 A|
|  34/5-1 A|
+----------+
only showing top 20 rows



In [18]:
#Filter

df[df.WELL =='16/10-1'].show(4)

+-------+------------+----------+---------+------------+------------+---------+------------+----+------------+------------+----+------------+----+----+----+------------+------------+------------+------------+----+------------+----+---------+----+----+------------+--------------------------------+---------------------------------+
|   WELL|    DEPTH_MD|     X_LOC|    Y_LOC|       Z_LOC|       GROUP|FORMATION|        CALI|RSHA|        RMED|        RDEP|RHOB|          GR| SGR|NPHI| PEF|         DTC|          SP|          BS|         ROP| DTS|        DCAL|DRHO|MUDWEIGHT|RMIC|ROPA|         RXO|FORCE_2020_LITHOFACIES_LITHOLOGY|FORCE_2020_LITHOFACIES_CONFIDENCE|
+-------+------------+----------+---------+------------+------------+---------+------------+----+------------+------------+----+------------+----+----+----+------------+------------+------------+------------+----+------------+----+---------+----+----+------------+--------------------------------+---------------------------------+
|16/

In [28]:
# Creating a new column
import pyspark.sql.functions as f


df.withColumn('cond',f.when((df.DEPTH_MD>500),1).otherwise('depth_less')).show(4)


+-------+--------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+------------+---------+----+----+----+--------------------------------+---------------------------------+----------+
|   WELL|DEPTH_MD|       X_LOC|    Y_LOC|       Z_LOC|       GROUP|FORMATION|        CALI|RSHA|        RMED|        RDEP|        RHOB|          GR| SGR|NPHI|         PEF|         DTC|          SP|  BS|         ROP| DTS|DCAL|        DRHO|MUDWEIGHT|RMIC|ROPA| RXO|FORCE_2020_LITHOFACIES_LITHOLOGY|FORCE_2020_LITHOFACIES_CONFIDENCE|      cond|
+-------+--------+------------+---------+------------+------------+---------+------------+----+------------+------------+------------+------------+----+----+------------+------------+------------+----+------------+----+----+------------+---------+----+----+----+--------------------------------+-----------------------